### Un poco más de estadística

* Chequear diferencias estadísticas entre las dimensiones.
    * En un mismo período y la misma across períodos.
    * Antes/después de normalizar.
* Chart de los traits.
    * Lineal chart.
    * Heatmap normalizado por trait.


* Normality test para lo del modelo.


Tests específicos para series de tiempo:

* Chequear series de tiempo con Friedman (un análisis multi de diferencias estadísticas).
* Chequear auto-correlación.
* Chequear si son estacionarias o no.
* No serial correlation.
* Durbin Watson
* Granger causality (efectos de grupo)
* Similarity de series de tiempo (para evaluar across phases)



In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
from collections import deque

In [ ]:
import scipy
from scipy.stats import mannwhitneyu
from scipy.stats import wilcoxon
from scipy.stats import brunnermunzel

from collections import deque

from statistics import mean, stdev
from math import sqrt

def cohens_d(c0,c1):
    cohens_d = abs((mean(c0) - mean(c1)) / (sqrt((stdev(c0) ** 2 + stdev(c1) ** 2) / 2)))
    if cohens_d < 0.1:
        return 'negligent'
    if cohens_d < 0.35:
        return 'small' 
    if cohens_d < 0.65:
        return 'medium'
    return 'large'

def unpaired_analysis(df,cols,alpha=0.01): # le pasamos un único df y calcula para todos los pares de columnas
    differents = deque()
    for i in range(0,len(cols)):
        for j in range(i+1,len(cols)):
            if scipy.stats.normaltest(df[cols[i]],nan_policy='omit').pvalue < alpha or scipy.stats.normaltest(df[cols[j]],nan_policy='omit').pvalue < alpha:
                statsb = mannwhitneyu(df[cols[i]].dropna(),df[cols[j]].dropna(),alternative='two-sided')
                statsg = mannwhitneyu(df[cols[i]].dropna(),df[cols[j]].dropna(),alternative='greater')
                statsl = mannwhitneyu(df[cols[i]].dropna(),df[cols[j]].dropna(),alternative='less')
                differents.append([cols[i],cols[j],statsb.pvalue,statsg.pvalue,statsl.pvalue,cohens_d(df[cols[i]].dropna(),df[cols[j]].dropna())])
            else: # both normal
                statsb = scipy.stats.ttest_ind(df[cols[i]],df[cols[j]],nan_policy='omit') 
                differents.append([cols[i],cols[j],statsb.statistic,statsb.pvalue,cohens_d(df[cols[i]].dropna(),df[cols[j]].dropna())])
    return differents
                
def unpaired_analysis_across(df,df1,cols,alpha=0.01): # le pasamos un único df y calcula para todos los pares de columnas
    differents = deque()
    for i in tqdm(range(0,len(cols))):
        
            if list(df[cols[i]].values) == list(df1[cols[i]].values):
                continue
        
            if scipy.stats.normaltest(df[cols[i]],nan_policy='omit').pvalue < alpha or scipy.stats.normaltest(df1[cols[i]],nan_policy='omit').pvalue < alpha:
                statsb = mannwhitneyu(df[cols[i]].dropna(),df1[cols[i]].dropna(),alternative='two-sided')
                statsg = mannwhitneyu(df[cols[i]].dropna(),df1[cols[i]].dropna(),alternative='greater')
                statsl = mannwhitneyu(df[cols[i]].dropna(),df1[cols[i]].dropna(),alternative='less')
                differents.append([cols[i],cols[i],statsb.pvalue,statsg.pvalue,statsl.pvalue,cohens_d(df[cols[i]].dropna(),df1[cols[i]].dropna())])
            else: # both normal
                statsb = scipy.stats.ttest_ind(df[cols[i]],df1[cols[i]],nan_policy='omit') 
                differents.append([cols[i],cols[i],statsb.statistic,statsb.pvalue,cohens_d(df[cols[i]].dropna(),df1[cols[i]].dropna())])
    return differents
      
def unpaired_analysis_bm(df,df1,cols,alpha=0.01): # le pasamos un único df y calcula para todos los pares de columnas
    differents = deque()
    for i in tqdm(range(0,len(cols))):
            if scipy.stats.normaltest(df[cols[i]],nan_policy='omit').pvalue < alpha or scipy.stats.normaltest(df1[cols[i]],nan_policy='omit').pvalue < alpha:
                statsb = brunnermunzel(df[cols[i]].dropna(),df1[cols[i]].dropna(),alternative='two-sided')
                statsg = brunnermunzel(df[cols[i]].dropna(),df1[cols[i]].dropna(),alternative='greater')
                statsl = brunnermunzel(df[cols[i]].dropna(),df1[cols[i]].dropna(),alternative='less')
                differents.append([cols[i],cols[i],statsb.pvalue,statsg.pvalue,statsl.pvalue,cohens_d(df[cols[i]].dropna(),df1[cols[i]].dropna())])
            else: # both normal
                statsb = scipy.stats.ttest_ind(df[cols[i]],df1[cols[i]],nan_policy='omit') 
                differents.append([cols[i],cols[i],statsb.statistic,statsb.pvalue,cohens_d(df[cols[i]].dropna(),df1[cols[i]].dropna())])
    return differents
    
def paired_analysis(df,df1,cols,alpha=0.01): # pasamos dos dfs y calcula para la misma columna en los dos dfs
    differents = deque()
    colsdf1 = set(df1.columns)
    for i in range(0,len(cols)):   
        
        print(cols[i], cols[i] in colsdf1)
        
        if cols[i] not in colsdf1:
            continue
        if len(df[cols[i]].dropna()) != len(df1[cols[i]].dropna()):
            continue
        if scipy.stats.normaltest(df[cols[i]],nan_policy='omit').pvalue < alpha or scipy.stats.normaltest(df1[cols[i]],nan_policy='omit').pvalue < alpha:
            statsb = wilcoxon(df[cols[i]].dropna(),df1[cols[i]].dropna(),alternative='two-sided',)
            statsg = wilcoxon(df[cols[i]].dropna(),df1[cols[i]].dropna(),alternative='greater')
            statsl = wilcoxon(df[cols[i]].dropna(),df1[cols[i]].dropna(),alternative='less')
            differents.append([cols[i],cols[i],statsb.pvalue,statsg.pvalue,statsl.pvalue,cohens_d(df[cols[i]].dropna(),df1[cols[i]].dropna())])
        else: # both normal
            statsb = scipy.stats.ttest_rel(df[cols[i]],df1[cols[i]],nan_policy='omit') 
            differents.append([cols[i],cols[i],statsb.statistic,statsb.pvalue,cohens_d(df[cols[i]].dropna(),df1[cols[i]].dropna())])
    
    return differents

# compara para todas las columnas dentro de un mismo df, mismos datos, diferentes tratamientos
def paired_analysis(df,cols,alpha=0.01): 
    
    differents = deque()
    for i in range(0,len(cols)):   
        
        for j in range(0,len(cols)):
            
            if i == j:
                continue
            
            print(cols[i],cols[j])
            
            if list(df[cols[i]].values) == list(df[cols[j]].values):
                continue
            
            if scipy.stats.normaltest(df[cols[i]],nan_policy='omit').pvalue < alpha or scipy.stats.normaltest(df[cols[j]],nan_policy='omit').pvalue < alpha:
                statsb = wilcoxon(df[cols[i]],df[cols[j]],alternative='two-sided',)
                statsg = wilcoxon(df[cols[i]],df[cols[j]],alternative='greater')
                statsl = wilcoxon(df[cols[i]],df[cols[j]],alternative='less')
                differents.append([cols[i],cols[j],statsb.pvalue,statsg.pvalue,statsl.pvalue,cohens_d(df[cols[i]].dropna(),df[cols[j]].dropna())])
            else: # both normal
                statsb = scipy.stats.ttest_rel(df[cols[i]],df[cols[j]],nan_policy='omit') 
                differents.append([cols[i],cols[j],statsb.statistic,statsb.pvalue,cohens_d(df[cols[i]].dropna(),df[cols[j]].dropna())])
    
    return differents

def normality_test(df,cols=None,alpha=0.01):
    if cols is None:
        cols = df.columns
    
    norms = {}
    for i in range(0,len(cols)): # hipótesis nula: la distribución es normal. pvalue < alpha se rechaza hipótesis nula
        norms[cols[i]] = scipy.stats.normaltest(df[cols[i]],nan_policy='omit').pvalue
    return norms

In [ ]:
# levantamos todos los dfs que nos interesan para una combinación de nombre

def load_dfs(dir_data, name_part):
    df_dict = {}

#     df_dict['df_all'] = pd.read_pickle(dir_data + 'df_merged__all__' + name_part + '.pickle').sort_values(by='created_at')
    
    df_dict['df_no_covid_2019'] = pd.read_pickle(dir_data + 'df_merged__no_covid_2019__' + name_part + '.pickle').sort_values(by='created_at')
    df_dict['df_no_covid'] = pd.read_pickle(dir_data + 'df_merged__no_covid__' + name_part + '.pickle').sort_values(by='created_at')
    df_dict['df_pre_covid'] = pd.read_pickle(dir_data + 'df_merged__pre_covid__' + name_part + '.pickle').sort_values(by='created_at')
    df_dict['df_during_covid'] = pd.read_pickle(dir_data + 'df_merged__during_covid__' + name_part + '.pickle').sort_values(by='created_at')
    df_dict['df_post_covid'] = pd.read_pickle(dir_data + 'df_merged__post_covid__' + name_part + '.pickle').sort_values(by='created_at')

    return df_dict

dir_data = './df_merged/'
name_part = 'normalized_perc_85__tweet_None__df_tweets_social_dimensions_model_simplified' # para facilitar generar los nombres
name_part = 'normalized_perc_85__tweet_day__df_tweets_social_dimensions_model_simplified' 
# name_part = 'normalized_perc_85_word_length__tweet_None__df_tweets_social_dimensions_model_simplified'
name_part = 'tweet_None__df_tweets_social_dimensions_model_simplified'
# name_part = 'tweet_day__df_tweets_social_dimensions_model_simplified'
# name_part = 'tweet_week__df_tweets_social_dimensions_model_simplified'

df_dict = load_dfs(dir_data, name_part)
for k,v in df_dict.items():
    print(k,len(v))

In [ ]:
ten_dims = ['knowledge','power','status','trust','support','similarity','identity','fun','conflict'] # 'romance'

In [ ]:
for k,df in df_dict.items():
    print(k, normality_test(df,ten_dims))

In [ ]:
# dentro de un mismo grupo, calcular si hay diferencias
listi = deque()

for k,df in tqdm(df_dict.items()):
    dicti = {}
#     stats = paired_analysis(df,cols=ten_dims)
    stats = paired_analysis(df,cols=['retweet_count','reply_count','favorite_count'])
    print(stats)
    for t in stats:
        if len(t) < 6: # normal -- tiene que dividir y chequear para ver cual queremos rejectear
            dicti[t[0]+'##'+t[1]] = t[4] if t[3]/2 < 0.01 and t[2] > 0 else '-'
        else:
            dicti[t[0]+'##'+t[1]] = t[5] if t[2] < 0.01 else '-'
    dicti['df'] = k
    listi.append(dicti)
    
df_significant = pd.DataFrame(listi).set_index('df').T
# df_significant.to_csv('_df_paired_statistical_significances__'+name_part+'.csv')
df_significant

In [ ]:
# unpaired analysis -- distintos datos, mismo tratamiento
# cada fila es una combinación de dfs y cada columna es una dimensión
listi = deque()
ll = list(df_dict.items())
for i in range(0,len(ll)):
    for j in range(i+1,len(ll)):
        print(ll[i][0] + '##' +  ll[j][0])
        dicti = {}
        stats = unpaired_analysis_across(ll[i][1],ll[j][1],cols=ten_dims)
#         stats = unpaired_analysis_bm(ll[i][1],ll[j][1],cols=ten_dims)
        print(stats)
        for t in stats:
            if len(t) < 6: # normal -- tiene que dividir y chequear para ver cual queremos rejectear
                dicti[t[0]] = t[4] if t[3]/2 < 0.01 and t[2] > 0 else '-'
            else:
                dicti[t[0]] = t[5] if t[2] < 0.01 else '-'
        dicti['df'] = ll[i][0] + '##' +  ll[j][0]  # necesita el nombre también!
        listi.append(dicti)
    
df_significant = pd.DataFrame(listi).set_index('df').T
df_significant.to_csv('_df_unpaired_statistical_significances_across_phases__'+name_part+'.csv')
df_significant

In [ ]:
# hacer un line chart con el df pero agrupando por día
# len(df[df[column] == 1]) / len(df[column]) # esto parece que haría lo del percentage
from sklearn import preprocessing

def get_representation(df_data,dims,period='M',how='median',norm=None):
    df = pd.DataFrame(df_data.to_dict())
    df['created_at'] = pd.to_datetime(df['created_at']).dt.to_period(period)
    if how != 'perc':
        df = df.groupby(by='created_at')[dims].agg(how)
    else:
        df = df.groupby(by='created_at')[ten_dims]
        df = df.sum() / df.count()
    df = df.T
    if norm is not None:
        if norm == 'columns':
            df = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(df), columns=df.columns,index=df.index)
        else:
            df = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(df.T), columns=df.T.columns,index=df.T.index).T
    return df

In [ ]:
for k in df_dict:
    print('----- ',k)
    dd = get_representation(df_dict[k],eng,period='D',how='sum',norm=None)
    print(dd.T.corr().to_dict())

* Similarity de series de tiempo (para evaluar across phases)

In [ ]:
# relación entre los grupos de dimensiones para una misma etapa

from scipy.stats import kruskal
from scipy.stats import friedmanchisquare


for k in df_dict:
    print('---',k)
    print(friedmanchisquare(df_dict[k]['knowledge'],df_dict[k]['power'],df_dict[k]['status'],
                 df_dict[k]['trust'],df_dict[k]['support'],df_dict[k]['identity'],
                 df_dict[k]['fun'],df_dict[k]['conflict'],df_dict[k]['romance'],
                 df_dict[k]['similarity']))
    print(kruskal(df_dict[k]['knowledge'],df_dict[k]['power'],df_dict[k]['status'],
                 df_dict[k]['trust'],df_dict[k]['support'],df_dict[k]['identity'],
                 df_dict[k]['fun'],df_dict[k]['conflict'],df_dict[k]['romance'],
                 df_dict[k]['similarity']))

In [ ]:
# dikey fuller test
from statsmodels.tsa.stattools import adfuller
adfuller(df_dict['df_all'][['knowledge']])

In [ ]:
# granger causality
from statsmodels.tsa.stattools import grangercausalitytests

df_ = df_dict['df_pre_covid']

grangercausalitytests(df_[['knowledge', 'power']], maxlag=[1,2,3,4,5,6])

In [ ]:
df_dict['df_no_covid']['knowledge'].values

In [ ]:
df_dict['df_during_covid']['knowledge'].values

In [ ]:
_dims = ['power','status','trust','support','similarity','identity','fun','conflict']
df_dict['df_post_covid'][ten_dims].plot()

In [ ]:
df_dict['df_during_covid'][ten_dims].plot()

In [ ]:
df_dict['df_pre_covid'][ten_dims].plot()

In [ ]:
df_dict['df_no_covid'][ten_dims].plot()

In [ ]:
pd.concat([df_dict['df_pre_covid'],df_dict['df_during_covid'],df_dict['df_post_covid']],axis=0)[['fun','similarity','identity']].plot()

In [ ]:
df_dict['df_no_covid'][ten_dims].describe()

In [ ]:
# dikey fuller test
from statsmodels.tsa.stattools import adfuller

for d in ten_dims:
    print(adfuller(df_dict['df_pre_covid'][[d]]))
    
from statsmodels.tsa.stattools import kpss
kpss(df_dict['df_during_covid'][['trust']])

In [ ]:
# granger causality -- lo que me interesa ver son los casos donde una no me sirve para la otra, dado que ahí "no habría relación"
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
import numpy as np

listi = []
for k in df_dict:
    df_ = df_dict[k].sort_values(by='created_at')
    dicti = {}
    print('-------------- ',k)
    for i in tqdm(range(0,len(ten_dims))):
        lag = kpss(df_[[ten_dims[i]]],nlags='legacy')[2] # 'legacy', 'auto' dan distintos resultados, MUY distintos
        print(lag)
        for j in range(0,len(ten_dims)):
            if i == j:
                continue
            dicti[ten_dims[i]+'##'+ten_dims[j]] = grangercausalitytests(df_[[ten_dims[i],ten_dims[j]]], maxlag=[lag],verbose=False)[lag][0]['params_ftest'][1]
            print(ten_dims[i],'--',ten_dims[j],grangercausalitytests(df_[[ten_dims[i],ten_dims[j]]], maxlag=[lag],verbose=False)[lag][0]['params_ftest'])
    dicti['df'] = k
    listi.append(dicti)
    
aa = pd.DataFrame(listi)
aa = aa.set_index('df').T
aa

In [ ]:
from tslearn.metrics import dtw
# https://cs.stackexchange.com/questions/53250/normalized-measure-from-dynamic-time-warping
# La similarity hay que normalizarla !!
# para la similarity, necesitaría que la diferencia entre las mismas dimensiones across phases sea mayor que las internas?
# si se da eso, es que son diferentes

dtw_score = dtw(df_dict['df_pre_covid']['knowledge'], df_dict['df_post_covid']['knowledge'])
dtw_score

# para normalizar es (M(x) - D(x,y)) / M(x) y M(x) = len(x) * range(x) -- NO es tan claro esto


In [ ]:
# Análisis de granger
name_part = 'tweet_None__df_tweets_social_dimensions_model_simplified'

df_granger = pd.read_pickle('__df_granger__'+name_part+'.pickle')
df_paired = pd.read_csv('_df_paired_statistical_significances__'+name_part+'.csv').rename(columns={'Unnamed: 0':'comb'}).set_index('comb')

In [ ]:
alpha = 0.01

ww = 'df_no_covid_2019'

aa = df_granger[[ww]].merge(df_paired[[ww]],left_index=True,right_index=True,suffixes=('_granger','_paired'))
aa[(aa[ww+'_granger'] > 0.01) & (aa[ww+'_paired'] != '-')]
aa[(aa[ww+'_granger'] < 0.01)].sort_index()

In [ ]:
pre_ww = 'df_no_covid'
ww = 'df_pre_covid'
df_granger[(df_granger[pre_ww] < alpha) & (df_granger[ww] > alpha)][[pre_ww,ww]] # si -> no
df_granger[(df_granger[pre_ww] < alpha) & (df_granger[ww] < alpha)][[pre_ww,ww]] # si -> si
df_granger[(df_granger[pre_ww] > alpha) & (df_granger[ww] < alpha)][[pre_ww,ww]] # no -> si
df_granger[(df_granger[pre_ww] < alpha) & (df_granger[ww] < alpha)][[pre_ww,ww]] # no -> no

In [ ]:
pre_ww = 'df_during_covid'
ww = 'df_post_covid'
df_paired[(df_paired[pre_ww] != '-') & (df_paired[ww] == '-')][[pre_ww,ww]] # si -> no
df_paired[(df_paired[pre_ww] != '-') & (df_paired[ww] != '-')][[pre_ww,ww]] # si -> si
df_paired[(df_paired[pre_ww] == '-') & (df_paired[ww] != '-')][[pre_ww,ww]] # no -> si